In [ ]:
from google.colab import drive
import os
import os
import cv2
import numpy as np
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import numpy as np
from google.colab.patches import cv2_imshow
import warnings
warnings.filterwarnings("ignore")
from skimage import data, segmentation, color
from skimage.future import graph

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
def load_data(path,images_to_load=50,resize=True,resize_dim=(512,512)):
  '''
  path : The path where the images are located.
  images_to_load : Maximum number of images to load.
  resize : Whether to resize the image.
  resize_dim : Specify dimensions for resizing, used only when resize=True.

  Return
  ------
  Loaded image and filename as a tuple for each entry.
  '''
  valid_images = [".png"]
  img_lst = []
  for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    img = cv2.imread(os.path.join(path,f))
    if resize:
      img = cv2.resize(img, resize_dim)
    img_lst.append([img,f])
  return img_lst



In [ ]:
melanoma_img_path = '/content/gdrive/MyDrive/CS_EE_577/Dataset/CentralView_labeled/Melanoma/'
non_melanoma_img_path = '/content/gdrive/MyDrive/CS_EE_577/Dataset/CentralView_labeled/Non_Melanoma/'

In [ ]:
m_lst = load_data(melanoma_img_path)
#nm_lst = load_data(non_melanoma_img_path)

In [ ]:
def perform_segmentation(img, n_segments):
  '''
  Segments the image and extracts out the lesion region using normalized cut.

  Parameters
  ----------
  img : The original input image.
  n_segments : The total number of segments for SLIC to generate.

  Returns
  -------
  Dictionary object comprising of:
  1. Original Image
  2. Bordered Image
  3. Mask
  4. Segmented Image
  '''
  original_img = img.copy()
  segments = slic(img, n_segments = n_segments, compactness=10, max_iter=10, sigma=2)
  bordered_img = mark_boundaries(img, segments)
  g = graph.rag_mean_color(img, segments, mode='similarity')
  res = graph.cut_normalized(segments, g)
  res = color.label2rgb(res, img, kind='avg', bg_label=0)
  grayscale = np.array(rgb2gray(res))
  grayscale[grayscale>0] = 255
  mask = grayscale.copy()
  r,g,b = cv2.split(original_img)
  r = np.where(mask!=0,r,mask)
  g = np.where(mask!=0,g,mask)
  b = np.where(mask!=0,b,mask)
  segmented_img=np.dstack((r,g,b))
  return {'Original Image': original_img,
          'Bordered Image': bordered_img,
          'Mask':mask,
          'Segmented Image': segmented_img}

def choose_nsegments(lst, segments_lst, skip_initial=0):
  '''
  Parameters
  ----------
  lst: lst of images.
  segments_lst: specifies the different n_segments to use for each SLIC iteration.
    While choosing segments, specify the index of the segment number.
  skip_initial: specify the number of initial images to skip. 

  Return
  ------
  The value of n_segments for each image. 
  '''
  n_segment_mapping = {}
  keys_list = np.arange(1,len(segments_lst)+1)
  zip_iterator = zip(keys_list, segments_lst)
  code_mapping = dict(zip_iterator)
  code_mapping[0] = None
  lst = lst[skip_initial:]  
  for img, name in lst:
    # Resizing for quicker loading
    img = cv2.resize(img, (240, 240))
    cv2_imshow(img)
    print('----------------------')
    for n_segments in segments_lst:
      res = perform_segmentation(img, n_segments)
      cv2_imshow(res['Segmented Image'])
      print('----------------------')
    res = input('Enter the n_segment code corresponding to the best segmentation:')
    n_segment_mapping[name] = code_mapping[int(res)]
  return n_segment_mapping

def store_image(img, path, f_name):
  '''
  Stores the image at the path specified.

  Parameters
  ----------
  img : The image to store.
  path : The path where the image is to be stored.
  f_name : Name of the file for storing.
  '''
  path = os.getcwd()+path
  try:
    os.makedirs(path)
  except:
    pass
  img = np.array(img)
  print(f'Stored {f_name} successfully!')
  cv2.imwrite(path+f_name, img)

def segment_and_store(lst, n_segment_mapping, dir_path):
  '''
  Segments the images in the list based on the mapping and
  stores them to the directory.
  '''
  resegment_lst = []
  segments_mask_lst = []
  for img, name in lst:
    if n_segment_mapping[name] is None:
      resegment_lst.append(name)
      continue
    res = perform_segmentation(img, n_segment_mapping[name])
    f_name = name.split('.')[0].split('_')[0]+'_SegmentedImage.png'
    store_image(res['Segmented Image'],dir_path,f_name)
  print('Stored Segments for all images!')
  return resegment_lst

# New section

In [ ]:
n_segment_mapping = choose_nsegments(nm_lst, [50,100,200,500], skip_initial=49)

In [ ]:
# Did a segment mapping before for first 49 samples.
prev_segment_mapping = {'0004_TotalFocus.png': 50,
 '0013_TotalFocus.png': 1000,
 '0015_TotalFocus.png': 1000,
 '0017_TotalFocus.png': 500,
 '0018_TotalFocus.png': 100,
 '0021_TotalFocus.png': 50,
 '0023_TotalFocus.png': 1000,
 '0042_TotalFocus.png': 1000,
 '0049_TotalFocus.png': 50,
 '0056_TotalFocus.png': 1000,
 '0057_TotalFocus.png': 500,
 '0058_TotalFocus.png': 500,
 '0059_TotalFocus.png': 50,
 '0060_TotalFocus.png': 50,
 '0066_TotalFocus.png': 500,
 '0067_TotalFocus.png': 100,
 '0070_TotalFocus.png': 50,
 '0075_TotalFocus.png': 50,
 '0091_TotalFocus.png': 50,
 '0092_TotalFocus.png': 1000,
 '0093_TotalFocus.png': 1000,
 '0094_TotalFocus.png': 1000,
 '0095_TotalFocus.png': 500,
 '0096_TotalFocus.png': 1000,
 '0097_TotalFocus.png': 500,
 '0104_TotalFocus.png': 1000,
 '0129_TotalFocus.png': 50,
 '0131_TotalFocus.png': 100,
 '0156_TotalFocus.png': 50,
 '0157_TotalFocus.png': 500,
 '0161_TotalFocus.png': 100,
 '0167_TotalFocus.png': 500,
 '0168_TotalFocus.png': 500,
 '0169_TotalFocus.png': 500,
 '0170_TotalFocus.png': 1000,
 '0173_TotalFocus.png': 1000,
 '0174_TotalFocus.png': 500,
 '0175_TotalFocus.png': 1000,
 '0177_TotalFocus.png': 500,
 '0178_TotalFocus.png': 1000,
 '0179_TotalFocus.png': 1000,
 '0185_TotalFocus.png': 50,
 '0198_TotalFocus.png': 1000,
 '0213_TotalFocus.png': 50,
 '0214_TotalFocus.png': 100,
 '0236_TotalFocus.png': 1000,
 '0244_TotalFocus.png': 1000,
 '0245_TotalFocus.png': 1000,
 '0246_TotalFocus.png': 500}

In [ ]:
n_segment_mapping.update(prev_segment_mapping)

In [ ]:
nm_resegment_lst = segment_and_store(nm_lst, n_segment_mapping, dir_path='/gdrive/MyDrive/CS_EE_577/Dataset/SegmentedImages_NormalizedCut/Non_Melanoma/')

In [ ]:
m_segment_mapping = choose_nsegments(m_lst, [50,100,200,500], skip_initial=0)

In [ ]:
melanoma_resegment_lst = segment_and_store(m_lst, m_segment_mapping, dir_path='/gdrive/MyDrive/CS_EE_577/Dataset/SegmentedImages_NormalizedCut/Melanoma/')

In [ ]:
melanoma_resegment_lst